### CONFIGURATION

In [1]:
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import numpy as np
import copy
import sys
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Hartwig imports
sys.path.append("/Users/peterpriestley/hmf/repos/scripts/analysisscripts") 
import analyseVCF as aVCF
import venn as vn

In [3]:
#CHROM SLICING
minChromFrac = 0
maxChromFrac = 26

In [44]:
# BED FILE
BED_PATH = "/Users/peterpriestley/hmf/analyses/giabTruthsets/70-30mixin/"
BED_FILE_NAME = "na12878-na24385-somatic-truth-regionsSORTED.bed"

# TRUTH SET
SAMPLE_NAMES_TRUTH = {'NA12878':'truth'}
VCF_PATH_TRUTH = "/Users/peterpriestley/hmf/analyses/giabTruthsets/70-30mixin/"
VCF_FILE_NAME_TRUTH = "na12878-na24385-somatic-truth.vcf"

# COMBINED VCF CONFIG
VCF_SAMPLE = "CPCT22222222"
VCF_PATH = "/Users/peterpriestley/hmf/analyses/70-30sample/170124_GIAB-mixin-v2_v1.11_Somatic_neoprep/"
VCF_FILE_NAME = VCF_SAMPLE + "R_"+ VCF_SAMPLE + "T_merged_somatics.vcf"#_somatic_filtered.vcf"#
#VCF_FILE_NAME = VCF_SAMPLE + "R_"+ VCF_SAMPLE + "T_melted.vcf"#_somatic_filtered.vcf"#
SAMPLE_NAMES = {VCF_SAMPLE + 'T.mutect':'mutect', VCF_SAMPLE + 'T.freebayes':'freebayes', \
                'TUMOR.strelka':'strelka', 'TUMOR.varscan':'varscan'}
#SAMPLE_NAMES = {VCF_SAMPLE + 'T':'melted'}

### Functions

In [5]:
def filterByChromFrac(df):
    return df[(df.chromFrac > minChromFrac)&(df.chromFrac < maxChromFrac)]

In [6]:
def calculateTruth(df,dfTruth):
    df = pd.merge(df,dfTruth,how='left', left_index=True,right_index=True,suffixes=('', '_Truth'))
    df['hasTP'] = False
    df['hasFP'] = False
    for columnName in list(df):
        if columnName.endswith('allele') and not columnName.startswith('truth'):
            df['hasTP'] = (df['hasTP']) | ((df[columnName[:-6]+'indelDiff'] == df['truthindelDiff']) \
                    & (~pd.isnull(df['truthindelDiff']) & (df['variantType'] == 'INDEL'))) |((df[columnName] == df['truthallele']) \
                    & (df['variantType'] == 'SNP'))
            df['hasFP'] = (df['hasFP']) | ((df[columnName[:-6]+'indelDiff'] != df['truthindelDiff']) \
                    & (df['variantType'] == 'INDEL') & (df[columnName[:-6]+'indelDiff'] != '')& (~pd.isnull(df['truthallele']))) |((df[columnName] != df['truthallele']) \
                    & (df['variantType'] == 'SNP') & (df[columnName] != '')& (~pd.isnull(df['truthallele'])))
    df['Truth'] = (df['hasTP']) &  (df['hasFP'] == False)
    return df

In [7]:
def calcuatePrecisionSensivityMatrix(df):
    outputdata = []
    for columnName in list(df):
        if columnName.endswith('allele') & ~columnName.endswith('truthallele'):
            myCaller = columnName[:-6]
            variantTypes = df[(df[myCaller+'allele'] != '')].variantType.unique()
            for variantType in variantTypes:
                truePositives = len(df[(df[myCaller+'allele'] != '') & (df['Truth'] == True) &(df['variantType'] == variantType)])
                positives = len(df[(df[myCaller+'allele'] != '')&(df['variantType'] == variantType)])
                truthSet = len(dfTruth[dfTruth['variantType'] == variantType]) 
                falseNegatives = truthSet - truePositives
                if positives > 0 and truthSet > 0:
                    outputdata.append([variantType, myCaller, truthSet,truePositives,positives-truePositives, falseNegatives, \
                                   round(truePositives/float(positives),4),round(truePositives/float(truthSet),4)])
    
    outputDF = pd.DataFrame(outputdata)
    outputDF.columns = (['variantType','caller','truthSet','truePositives','falsePositives','falseNegatives','precision','sensitivity'])
    return outputDF.sort_values(['variantType','caller'])

<h3> Load VCFs and Prepare DF

In [25]:
## LOAD TRUTH SET VCF
bed = aVCF.loadBEDFile(BED_PATH,BED_FILE_NAME)
dfTruth = aVCF.loadVariantsFromVCF(VCF_PATH_TRUTH,VCF_FILE_NAME_TRUTH, \
                                   SAMPLE_NAMES_TRUTH,"Mix-in Truth Set",True,True,bed)
dfTruth = filterByChromFrac(dfTruth)
dfTruth = dfTruth[['chrom','pos','variantType','ref','truthallele','truthindelDiff','bedRegion']]
dfTruth = dfTruth.set_index(['chrom','pos'])

reading vcf file: na12878-na24385-somatic-truth.vcf
reading VCF File line: 1
reading VCF File line: 200001
reading VCF File line: 400001
reading VCF File line: 600001
reading VCF File line: 800001
reading VCF File line: 1000001
Number variants loaded: 1104460


In [45]:
# LOAD SAMPLE VCF + match to truth set
bed = aVCF.loadBEDFile(BED_PATH,BED_FILE_NAME)
dfProd = aVCF.loadVariantsFromVCF(VCF_PATH,VCF_FILE_NAME,SAMPLE_NAMES,VCF_SAMPLE,True,True,bed)
dfProd = filterByChromFrac(dfProd)
dfProd = dfProd.set_index(['chrom','pos'])
dfProd = calculateTruth(dfProd,dfTruth)

reading vcf file: CPCT22222222R_CPCT22222222T_merged_somatics.vcf
reading VCF File line: 1
reading VCF File line: 200001
reading VCF File line: 400001
reading VCF File line: 600001
reading VCF File line: 800001
reading VCF File line: 1000001
reading VCF File line: 1200001
reading VCF File line: 1400001
Number variants loaded: 1098474


### PRECISION + SENSITIVITY|

In [46]:
outputDF = calcuatePrecisionSensivityMatrix(dfProd)
outputDF

,variantType,caller,truthSet,truePositives,falsePositives,falseNegatives,precision,sensitivity
6,INDEL,freebayes,96616,80905,721,15711,0.9912,0.8374
1,INDEL,strelka,96616,53338,136,43278,0.9975,0.5521
3,INDEL,varscan,96616,71015,719,25601,0.9900,0.7350
5,SNP,freebayes,1007844,991791,1471,16053,0.9985,0.9841
4,SNP,mutect,1007844,908928,8048,98916,0.9912,0.9019
0,SNP,strelka,1007844,945591,3971,62253,0.9958,0.9382
2,SNP,varscan,1007844,926099,852,81745,0.9991,0.9189


In [47]:
dftemp2 = dfProd.reset_index()
pd.pivot_table(dftemp2, values='pos', index=['numCallers','vennSegment'], columns=['variantType','Truth'], aggfunc='count')

variantType                           INDEL          MIXED     SNP          
Truth                                 False    True  False   False     True 
numCallers vennSegment                                                      
1          freebayes                  417.0  10510.0   NaN   864.0   14338.0
           mutect                       NaN      NaN   NaN  6081.0    3155.0
           strelka                     57.0    427.0   NaN  1969.0     731.0
           varscan                    442.0   1863.0   NaN   431.0    1496.0
2          freebayes-mutect             NaN      NaN   1.0   133.0   14547.0
           strelka-freebayes           37.0   1911.0  35.0    63.0   12019.0
           strelka-mutect               NaN      NaN   NaN  1563.0    2919.0
           strelka-varscan             10.0    668.0   NaN    61.0     536.0
           varscan-freebayes          235.0  18152.0  10.0   103.0   14463.0
           varscan-mutect               NaN      NaN   NaN    16.0     208.0
3          strelka-freebayes-mutect     NaN      NaN   NaN    98.0   27903.0
           strelka-varscan-freebayes   32.0  50332.0   NaN    84.0   49200.0
           strelka-varscan-mutect       NaN      NaN   NaN    31.0     875.0
           varscan-freebayes-mutect     NaN      NaN   NaN    24.0    7913.0
           varscan-strelka-freebayes    NaN      NaN   1.0     NaN       NaN
4          Intersection                 NaN      NaN   NaN   102.0  851408.0

In [48]:
pd.pivot_table(dftemp2[(dftemp2.variantType=='INDEL')|(dftemp2.variantType=='SNP')], values='pos', index=['numCallers'], columns=['variantType','Truth'], aggfunc='count')

variantType  INDEL              SNP          
Truth        False    True    False     True 
numCallers                                   
1            916.0  12800.0  9345.0   19720.0
2            282.0  20731.0  1939.0   44692.0
3             32.0  50332.0   237.0   85891.0
4              NaN      NaN   102.0  851408.0

<h3> SNP

In [ ]:
snpdf = dfProd[(dfProd.variantType == 'SNP')]

In [ ]:
vn.venn([snpdf[snpdf.mutectallele != '']['chromPos'], \
         snpdf[snpdf.strelkaallele != '']['chromPos'], \
        snpdf[snpdf.freebayesallele != '']['chromPos'], \
        snpdf[snpdf.varscanallele != '']['chromPos'] \
        ],['mutect','strelka','freebayes','varscan'],figsize=(6,6))

<h3> Allelic Depth

In [ ]:
#Alllele Freq By Caller
for columnName in list(dfProd):
    if columnName.endswith('freebayesAF'):
        print columnName
        ser = dfProd[(dfProd.Truth == True)][columnName[:-2] + 'AF']
        ser = ser.sort_values()
        #ser[len(ser)] = ser.iloc[-1]
        cum_dist = np.linspace(0.,1.,len(ser))
        ser_cdf = pd.Series(cum_dist, index=ser,name=columnName[:-2]+": c="+str(ser.count())+" m="+str(round(ser.median(),2)))
        ser_cdf.plot(drawstyle='steps',legend=True,title=" Allelic Frequency by Caller (AllelicFreq > 0.0)",figsize=[15,6],xlim=[0,1])

In [ ]:
truths = [True,False]
for truth in truths:
    #Alllele Freq By Caller
    for columnName in list(dfProd):
        if columnName.endswith('freebayesQS') and not columnName.endswith('lQS'):
            ser = dfProd[(dfProd.Truth == truth)][columnName]
            ser = ser.sort_values()
            cum_dist = np.linspace(0.,1.,len(ser))
            ser_cdf = pd.Series(cum_dist, index=ser,name=columnName[:-2]+": c="+str(ser.count())+ " "+ str(truth) +" m="+str(round(ser.median(),2)))
            ser_cdf.plot(drawstyle='steps',legend=True,title=" Allelic Frequency by Caller (AllelicFreq > 0.0)",figsize=[15,6],xlim=[0,100],ylim=[0,0.5])

### SCRATCH

In [ ]:
dfProd[(dfProd.variantType=='INDEL')&(dfProd.Truth==False)&(dfProd.freebayesallele<>'')&(dfProd.chromPos.str.contains('1:'))].head(20)

In [36]:
dftemp[(dftemp.vennSegment.str.contains('mutect'))&(dftemp.Truth==False)].head(200)

,chrom,pos,chromPos,chromFrac,id,ref,vennSegment,numCallers,variantType,variantSubType,filter,bedRegion,inDBSNP,inCOSMIC,annGene,annWorstImpact,annWorstEffect,annAllEffects,consensus,meltedallele,meltedAF,meltedDP,meltedQS,meltedSGT,meltedindelDiff,meltedSVLenMin,meltedSVLenMax,meltedSVStartMin,meltedSVStartMax,patientName,variantType_Truth,ref_Truth,truthallele,truthindelDiff,bedRegion_Truth,hasTP,hasFP,Truth
67,1,101064802,1:101064802,1.405475,rs201312805,A,mutect,1,SNP,,PASS,Default,True,False,,,,,False,C,0.049020,102,-1.0,0/1,,,,,,CPCT11111111,NaN,NaN,NaN,NaN,NaN,False,False,False
71,1,101100440,1:101100440,1.405618,.,A,mutect,1,SNP,,PASS,Default,False,False,RP11-84O12.4,MODIFIER,intron_variant,intron_variant,False,T,0.059829,118,-1.0,0/1,,,,,,CPCT11111111,NaN,NaN,NaN,NaN,NaN,False,False,False
181,1,101706145,1:101706145,1.408048,.,A,mutect,1,SNP,,PASS,Default,False,False,S1PR1,MODIFIER,3_prime_UTR_variant,3_prime_UTR_variant,False,C,0.160920,88,-1.0,0/1,,,,,,CPCT11111111,NaN,NaN,NaN,NaN,NaN,False,False,False
439,1,102094475,1:102094475,1.409606,.,C,mutect,1,SNP,,PASS,Default,False,False,,,,,False,A,0.057143,105,-1.0,0/1,,,,,,CPCT11111111,NaN,NaN,NaN,NaN,NaN,False,False,False
445,1,102164622,1:102164622,1.409887,.,T,mutect,1,SNP,,PASS,Default,False,False,,,,,False,C,0.081081,75,-1.0,0/1,,,,,,CPCT11111111,NaN,NaN,NaN,NaN,NaN,False,False,False
446,1,102164623,1:102164623,1.409887,.,C,mutect,1,SNP,,PASS,Default,False,False,,,,,False,T,0.069444,74,-1.0,0/1,,,,,,CPCT11111111,NaN,NaN,NaN,NaN,NaN,False,False,False
454,1,10227866,1:10227866,1.041034,.,G,mutect,1,SNP,,PASS,Default,False,False,UBE4B,MODIFIER,intron_variant,intron_variant|intron_variant|intron_variant|i...,False,A,0.048193,84,-1.0,0/1,,,,,,CPCT11111111,NaN,NaN,NaN,NaN,NaN,False,False,False
587,1,102539345,1:102539345,1.411391,.,C,mutect,1,SNP,,PASS,Default,False,False,,,,,False,T,0.038760,130,-1.0,0/1,,,,,,CPCT11111111,NaN,NaN,NaN,NaN,NaN,False,False,False
967,1,102977866,1:102977866,1.413150,.,A,mutect,1,SNP,,PASS,Default,False,False,,,,,False,T,0.042017,120,-1.0,0/1,,,,,,CPCT11111111,NaN,NaN,NaN,NaN,NaN,False,False,False
1263,1,104578215,1:104578215,1.419571,.,G,mutect,1,SNP,,PASS,Default,False,False,,,,,False,T,0.112150,107,-1.0,0/1,,,,,,CPCT11111111,NaN,NaN,NaN,NaN,NaN,False,False,False


In [74]:
pd.set_option('display.max_rows', 500)
dftemp[((dftemp.vennSegment.str.contains('mutect'))|(dftemp.vennSegment=='Intersection'))&(dftemp.Truth==True)].head(40)

,chrom,pos,chromPos,chromFrac,id,ref,vennSegment,numCallers,variantType,variantSubType,filter,bedRegion,inDBSNP,inCOSMIC,annGene,annWorstImpact,annWorstEffect,annAllEffects,consensus,meltedallele,meltedAF,meltedDP,meltedQS,meltedSGT,meltedindelDiff,meltedSVLenMin,meltedSVLenMax,meltedSVStartMin,meltedSVStartMax,patientName,variantType_Truth,ref_Truth,truthallele,truthindelDiff,bedRegion_Truth,hasTP,hasFP,Truth
0,1,100028229,1:100028229,1.401316,rs72719698,G,Intersection,4,SNP,,PASS,Default,True,False,,,,,True,T,0.188679,106,-1.0,0/1,,,,,,CPCT11111111,SNP,G,T,,Default,True,False,True
2,1,100039519,1:100039519,1.401361,rs1890753,G,Intersection,4,SNP,,PASS,Default,True,False,,,,,True,C,0.327273,112,-1.0,0/1,,,,,,CPCT11111111,SNP,G,C,,Default,True,False,True
3,1,10004347,1:10004347,1.040138,rs61227057,C,Intersection,4,SNP,,PASS,Default,True,False,RP11-84A14.4,MODIFIER,intron_variant,intron_variant|intron_variant|intron_variant|i...,True,T,0.140187,107,-1.0,0/1,,,,,,CPCT11111111,SNP,C,T,,Default,True,False,True
4,1,100045239,1:100045239,1.401384,rs11166276,C,Intersection,4,SNP,,PASS,Default,True,False,,,,,True,T,0.284211,97,-1.0,0/1,,,,,,CPCT11111111,SNP,C,T,,Default,True,False,True
5,1,100046246,1:100046246,1.401388,rs6702619,T,Intersection,4,SNP,,PASS,Default,True,False,,,,,True,G,0.271028,108,-1.0,0/1,,,,,,CPCT11111111,SNP,T,G,,Default,True,False,True
6,1,100049648,1:100049648,1.401402,rs7543039,C,Intersection,4,SNP,,PASS,Default,True,False,,,,,True,T,0.303922,105,-1.0,0/1,,,,,,CPCT11111111,SNP,C,T,,Default,True,False,True
7,1,100049785,1:100049785,1.401402,rs7543130,C,Intersection,4,SNP,,PASS,Default,True,False,,,,,True,A,0.295652,116,-1.0,0/1,,,,,,CPCT11111111,SNP,C,A,,Default,True,False,True
8,1,10013014,1:10013014,1.040172,rs7516611,A,Intersection,4,SNP,,PASS,Default,True,False,NMNAT1,MODIFIER,intron_variant,intron_variant|intron_variant|intron_variant|i...,True,G,0.137931,89,-1.0,0/1,,,,,,CPCT11111111,SNP,A,G,,Default,True,False,True
10,1,100151835,1:100151835,1.401812,rs41285726,G,Intersection,4,SNP,,PASS,Default,True,False,PALMD,LOW,sequence_feature,sequence_feature|intron_variant|intron_variant...,True,A,0.159664,119,-1.0,0/1,,,,,,CPCT11111111,SNP,G,A,,Default,True,False,True
11,1,10016335,1:10016335,1.040186,rs34011816,C,Intersection,4,SNP,,PASS,Default,True,False,NMNAT1,MODIFIER,intron_variant,intron_variant|intron_variant|intron_variant|i...,True,A,0.147059,70,-1.0,0/1,,,,,,CPCT11111111,SNP,C,A,,Default,True,False,True


In [75]:
dftemp2[((dftemp2.vennSegment.str.contains('mutect'))|(dftemp2.vennSegment=='Intersection'))&(dftemp2.Truth==True)].head(40)

,chrom,pos,chromPos,chromFrac,id,ref,vennSegment,numCallers,variantType,variantSubType,filter,bedRegion,inDBSNP,inCOSMIC,annGene,annWorstImpact,annWorstEffect,annAllEffects,consensus,strelkaallele,strelkaAF,strelkaDP,strelkaQS,strelkaSGT,strelkaindelDiff,strelkaSVLenMin,strelkaSVLenMax,strelkaSVStartMin,strelkaSVStartMax,varscanallele,varscanAF,varscanDP,varscanQS,varscanSGT,varscanindelDiff,varscanSVLenMin,varscanSVLenMax,varscanSVStartMin,varscanSVStartMax,mutectallele,mutectAF,mutectDP,mutectQS,mutectSGT,mutectindelDiff,mutectSVLenMin,mutectSVLenMax,mutectSVStartMin,mutectSVStartMax,freebayesallele,freebayesAF,freebayesDP,freebayesQS,freebayesSGT,freebayesindelDiff,freebayesSVLenMin,freebayesSVLenMax,freebayesSVStartMin,freebayesSVStartMax,patientName,variantType_Truth,ref_Truth,truthallele,truthindelDiff,bedRegion_Truth,hasTP,hasFP,Truth
0,1,100028229,1:100028229,1.401316,rs72719698,G,Intersection,4,SNP,,PASS,Default,True,False,,,,,True,T,0.272,128,42,ref,,,,,,T,0.2703,112,36,2,,,,,,T,0.267,121,-1,ref-het,,,,,,T,0.267176,132,715.16,SNP,,,,,,CPCT22222222,SNP,G,T,,Default,True,False,True
2,1,100039519,1:100039519,1.401361,rs1890753,G,Intersection,4,SNP,,PASS,Default,True,False,,,,,True,C,0.349057,108,51,ref,,,,,,C,0.3333,102,46,2,,,,,,C,0.333,106,-1,ref-het,,,,,,C,0.348624,110,983.04,SNP,,,,,,CPCT22222222,SNP,G,C,,Default,True,False,True
3,1,10004347,1:10004347,1.040138,rs61227057,C,Intersection,4,SNP,,PASS,Default,True,False,,,,,True,T,0.107843,104,16,ref,,,,,,T,0.1111,99,13,2,,,,,,T,0.086,93,-1,ref-het,,,,,,T,0.113208,106,42.61,SNP,,,,,,CPCT22222222,SNP,C,T,,Default,True,False,True
4,1,100045239,1:100045239,1.401384,rs11166276,C,Intersection,4,SNP,,PASS,Default,True,False,,,,,True,T,0.426087,118,47,ref,,,,,,T,0.4273,110,52,2,,,,,,T,0.44,109,-1,ref-het,,,,,,T,0.446281,121,1343.74,SNP,,,,,,CPCT22222222,SNP,C,T,,Default,True,False,True
5,1,100046246,1:100046246,1.401388,rs6702619,T,Intersection,4,SNP,,PASS,Default,True,False,,,,,True,G,0.533333,109,61,ref,,,,,,G,0.54,100,85,2,,,,,,G,0.536,98,-1,ref-het,,,,,,G,0.541284,110,1672.24,SNP,,,,,,CPCT22222222,SNP,T,G,,Default,True,False,True
6,1,100049648,1:100049648,1.401402,rs7543039,C,Intersection,4,SNP,,PASS,Default,True,False,,,,,True,T,0.41,104,51,ref,,,,,,T,0.4,95,52,2,,,,,,T,0.37,92,-1,ref-het,,,,,,T,0.435185,108,1159.05,SNP,,,,,,CPCT22222222,SNP,C,T,,Default,True,False,True
7,1,100049785,1:100049785,1.401402,rs7543130,C,Intersection,4,SNP,,PASS,Default,True,False,,,,,True,A,0.431373,109,74,ref,,,,,,A,0.4227,97,67,2,,,,,,A,0.37,93,-1,ref-het,,,,,,A,0.433628,114,1223.69,SNP,,,,,,CPCT22222222,SNP,C,A,,Default,True,False,True
8,1,10013014,1:10013014,1.040172,rs7516611,A,Intersection,4,SNP,,PASS,Default,True,False,,,,,True,G,0.16092,90,19,ref,,,,,,G,0.1687,83,15,2,,,,,,G,0.163,88,-1,ref-het,,,,,,G,0.175824,93,246.58,SNP,,,,,,CPCT22222222,SNP,A,G,,Default,True,False,True
10,1,100151835,1:100151835,1.401812,rs41285726,G,Intersection,4,SNP,,PASS,Default,True,False,,,,,True,A,0.1875,98,29,ref,,,,,,A,0.1868,91,20,2,,,,,,A,0.167,90,-1,ref-het,,,,,,A,0.19,100,301.9,SNP,,,,,,CPCT22222222,SNP,G,A,,Default,True,False,True
11,1,10016335,1:10016335,1.040186,rs34011816,C,Intersection,4,SNP,,PASS,Default,True,False,,,,,True,A,0.333333,92,35,ref,,,,,,A,0.2973,74,21,2,,,,,,A,0.365,86,-1,ref-het,,,,,,A,0.350515,98,531.09,SNP,,,,,,CPCT22222222,SNP,C,A,,Default,True,False,True


In [70]:
len(dftemp)

1084072

In [71]:
len(dftemp2)

1098474